In [75]:
import bokeh
from bokeh import events
from bokeh.plotting import Figure, output_notebook, show
import numpy as np
import math
from scipy.integrate import solve_ivp
from bokeh.layouts import row, column
from bokeh.models import CustomJS, ColumnDataSource, PreText, TextInput
output_notebook()

Loading BokehJS ...

In [93]:
def bk_repr_app(doc):
    #Variables

    n = 2 #Hill coefficient
    Km = 40 #monomeres per cell
    beta = 0.2
    alpha0 = 0.2164 
    alpha = 216.4

    CmtetR0 = 1
    Cmlambda0 = 4
    CmlacI0 = 1
    CptetR0 = 2
    Cplambda0 = 1
    CplacI0 = 1
    
    #Figure settings

    fig = Figure(plot_width=700, plot_height=500, title="Repressilator model",x_axis_label='Time/mRNA half-life time', y_axis_label='Protein concentration, in units of Km')

    #Calculating data for plots
    
    def repressilator_model(t, data, alpha0, alpha, beta):
        m = data[:3]
        p = data[3:]
        dm0 = - m[0] + alpha/(1+p[2]**n) + alpha0
        dm1 = - m[1] + alpha/(1+p[0]**n) + alpha0
        dm2 = - m[2] + alpha/(1+p[1]**n) + alpha0
        dp0 = - beta*(p[0] - m[0])
        dp1 = - beta*(p[1] - m[1])
        dp2 = - beta*(p[2] - m[2])
        return [dm0,dm1,dm2,dp0,dp1,dp2]
    
    t0 = 0.0
    tf = 100.0
    time = np.linspace(0.0,100.0,1000)
    initval = np.array([CmtetR0,Cmlambda0,CmlacI0,CptetR0,Cplambda0,CplacI0])
    res = solve_ivp(repressilator_model, (t0,tf), initval, method='RK45', t_eval=time, args=(alpha0,alpha,beta))
    res = dict(res)
    time = res['t']
    y = res['y']
    source = ColumnDataSource(data=dict(x=time, ylacI=y[5], ytetR=y[3], ylambda=y[4]))
    fig.line('x','ylacI', source=source, color = 'blue', legend_label="lacI", width=3)
    fig.line('x','ytetR', source=source, color = 'green', legend_label="tetR", width=3)
    fig.line('x','ylambda', source=source, color = 'red', legend_label="lambda", width=3)
    
    def callback(attr, old, new):
        beta = float(Ibeta.value)
        alpha0 = float(Ialpha0.value)
        alpha = float(Ialpha.value)
        CmtetR0 = float()
        Cmlambda0 = float()
        CmlacI0 = float(ICmlacI0.value)
        CptetR0 = float(ICptetR0.value)
        Cplambda0 = float(ICptetR0.value)
        CplacI0 = float(ICplacI0.value)
        time = np.linspace(0.0,100.0,1000)
        initval = np.array([CmtetR0,Cmlambda0,CmlacI0,CptetR0,Cplambda0,CplacI0])
        res = solve_ivp(repressilator_model, (t0,tf), initval, method='RK45', t_eval=time, args=(alpha0,alpha,beta))
        res = dict(res)
        time = res['t']
        y = res['y']
        source.data = dict(x=time, ylacI=y[5], ytetR=y[3], ylambda=y[4])
        fig.renderers = []
        fig.line('x','ylacI', source=source, color = 'blue', legend_label="lacI", width=3)
        fig.line('x','ytetR', source=source, color = 'green', legend_label="tetR", width=3)
        fig.line('x','ylambda', source=source, color = 'red', legend_label="lambda", width=3)
        
    
    #Textboxes

    Ibeta = TextInput(value=str(beta))
    Tbeta = PreText(text = "beta - default value is " + str(beta))

    Ialpha0 = TextInput(value=str(alpha0))
    Talpha0 = PreText(text = "alpha0 - default value is " + str(alpha0))

    Ialpha = TextInput(value=str(alpha))
    Talpha = PreText(text = "alpha - default value is " + str(alpha))

    ICmlacI0 = TextInput(value=str(CmlacI0))
    TCmlacI0 = PreText(text = "CmlacI0 - default value is " + str(CmlacI0))

    ICmtetR0 = TextInput(value=str(CmtetR0))
    TCmtetR0 = PreText(text = "CmtetR0 - default value is " + str(CmtetR0))

    ICmlambda0 = TextInput(value=str(Cmlambda0))
    TCmlambda0 = PreText(text = "Cmlambda0 - default value is " + str(Cmlambda0))

    ICplacI0 = TextInput(value=str(CplacI0))
    TCplacI0 = PreText(text = "CplacI0 - default value is " + str(CplacI0))

    ICptetR0 = TextInput(value=str(CptetR0))
    TCptetR0 = PreText(text = "CptetR0 - default value is " + str(CptetR0))

    ICplambda0 = TextInput(value=str(Cplambda0))
    TCplambda0 = PreText(text = "Cplambda0 - default value is " + str(Cplambda0))
    
    #Callback triggers

    Ibeta.on_change('value', callback)
    Ialpha0.on_change('value', callback)
    Ialpha.on_change('value', callback)
    ICmlacI0.on_change('value', callback)
    ICmtetR0.on_change('value', callback)
    ICmlambda0.on_change('value', callback)
    ICplacI0.on_change('value', callback)
    ICptetR0.on_change('value', callback)
    ICplambda0.on_change('value', callback)
    
    
    
    #Legend settings

    fig.legend.location = "top_left"
    fig.xaxis.axis_label_text_font_size = '20pt'
    fig.yaxis.axis_label_text_font_size = '20pt'
    doc.add_root(row(fig,column(Ibeta,Tbeta,Ialpha0,Talpha0,Ialpha,Talpha,ICmlacI0,TCmlacI0,ICmtetR0,TCmtetR0,ICmlambda0,TCmlambda0,ICplacI0,TCplacI0,ICptetR0,TCptetR0,ICplambda0,TCplambda0)))

In [94]:
show(bk_repr_app)